In [4]:
from rx import of, operators as op, just, interval, from_, create
from rx.subject import Subject, ReplaySubject, BehaviorSubject
from typing import List


energy_threshold = 40

class Buffer():
    def __init__(self, val):
        self.val = val
    
    def __repr__(self):
        return f"buf: {self.val}"
    
    def __add__(self, other):
        self.val += other.val
        return self
    
def debug(name: str, enable=True):
    if enable:
        return op.do_action(lambda val: print(f"{name}: {val}"))
    else:
        return op.do_action(nop)

def print_res(val):
    print(f"res ---> {val}")
    
def nop(*val):
    pass
    
    
def sum_window(window: List[Buffer]):
    return sum(map(lambda buffer: buffer.val, window))

def avg_window(window: List[Buffer]):
    return sum_window(window) / len(window)

def is_low(window: List[Buffer]):
#     print(f"***low {window}: {avg_window(window) <= energy_threshold}")
    return avg_window(window) <= energy_threshold    

def is_high(window: List[Buffer]):
#     print(f"**high {window} : {avg(window) > energy_threshold}")
    return avg_window(window) > energy_threshold

def overlapping_windows_to_buffer_array(windows: List[List[Buffer]]) -> List[Buffer]:
    return list(map(lambda win: win[0], windows[:-1])) + windows[-1]

print("ok")

ok


In [1]:
from sr.audio.source.microphone import Microphone
source = Microphone()

2021-08-04 01:46:07.720 | INFO     | sr.audio.source.microphone:__init__:35 - pyaudio loaded


In [ ]:
from rx import of, operators as op, just, interval, from_, create
from rx.subject import Subject, ReplaySubject, BehaviorSubject
from sr.audio.recorder import get_speech_recordings_stream, AudioEnergyChangeDetection, _overlapping_windows_to_audio_segment, AudioEnergyEnum
from sr.audio.recording_config import RecordingConfig
from sr.rx.rx_utils import rx_debug, rx_print
from typing import List, Any
from pydub import AudioSegment


class AudioEnergyChangeDetection():
    CURRENT_BUFFER_ENERGY = AudioEnergyEnum.LOW
    DEBUG = False
    CURRENT_CHUNK = []

    @classmethod
    def energy_change_operator(cls, source):
        def high_energy_change_observable(observer, _):
            def analyze(window: List[AudioSegment]):
                previous_buffer_energy = cls.CURRENT_BUFFER_ENERGY
                cls.CURRENT_CHUNK.append(window)
                audio_segment = sum(window)
                # print(audio_segment, audio_segment.duration_seconds)

                if audio_segment.dBFS >= RecordingConfig.MINIMUM_DBFS:
                    cls.CURRENT_BUFFER_ENERGY = AudioEnergyEnum.HIGH
                else:
                    cls.CURRENT_BUFFER_ENERGY = AudioEnergyEnum.LOW

                if cls.DEBUG:
                    logger.info(f"current buf energy: {audio_segment.dBFS} {cls.CURRENT_BUFFER_ENERGY}")

                if previous_buffer_energy != cls.CURRENT_BUFFER_ENERGY and cls.CURRENT_BUFFER_ENERGY == AudioEnergyEnum.LOW:
                    observer.on_next(cls.CURRENT_CHUNK)
                    cls.CURRENT_CHUNK = []

            return source.subscribe(analyze, observer.on_error, observer.on_completed)

        return create(high_energy_change_observable).pipe(rx_debug("energy_change"))

from sr.audio.source.microphone import Microphone
source = Microphone()
windows = source.make_observable().pipe(
    op.buffer_with_count(RecordingConfig.WINDOW_SIZE, 1),
#     op.publish(),
#     op.ref_count()
    op.share()
)

# windows2 = ReplaySubject(10)
# windows2 = ReplaySubject().pipe(rx_debug("windows 2"))

# windows.subscribe(windows2)
# windows2 = windows.pipe(op.share())

energy_change_obs = windows.pipe(AudioEnergyChangeDetection.energy_change_operator, rx_debug("energy_change"), op.share())
# energy_change_obs.subscribe(print_res, print)
recording_window_openings = energy_change_obs.pipe(op.filter(lambda energy: energy == AudioEnergyEnum.HIGH))
recording_window_closings = energy_change_obs.pipe(op.filter(lambda energy: energy == AudioEnergyEnum.LOW))

print("start")
windows.pipe(
#     rx_debug("windows"),
    AudioEnergyChangeDetection.energy_change_operator,
#     op.buffer_toggle(energy_change_obs, lambda _: recording_window_closings),
    op.map(_overlapping_windows_to_audio_segment)
).subscribe(rx_print, print)

2021-08-04 01:54:48.376 | INFO     | sr.audio.source.microphone:__init__:35 - pyaudio loaded


start


2021-08-04 01:54:49.970 | INFO     | sr.rx.rx_utils:<lambda>:9 - energy_change: [[<pydub.audio_segment.AudioSegment object at 0x000001CB7F6E6160>, <pydub.audio_segment.AudioSegment object at 0x000001CB7F6E6908>, <pydub.audio_segment.AudioSegment object at 0x000001CB7F6E6978>, <pydub.audio_segment.AudioSegment object at 0x000001CB7F6E6D68>, <pydub.audio_segment.AudioSegment object at 0x000001CB7F19E9B0>], [<pydub.audio_segment.AudioSegment object at 0x000001CB7F6E6908>, <pydub.audio_segment.AudioSegment object at 0x000001CB7F6E6978>, <pydub.audio_segment.AudioSegment object at 0x000001CB7F6E6D68>, <pydub.audio_segment.AudioSegment object at 0x000001CB7F19E9B0>, <pydub.audio_segment.AudioSegment object at 0x000001CB7F6E60B8>], [<pydub.audio_segment.AudioSegment object at 0x000001CB7F6E6978>, <pydub.audio_segment.AudioSegment object at 0x000001CB7F6E6D68>, <pydub.audio_segment.AudioSegment object at 0x000001CB7F19E9B0>, <pydub.audio_segment.AudioSegment object at 0x000001CB7F6E60B8>, <pyd

In [1]:
# Test 2
# buffer = list(map(Buffer, [0, 0, 42, 43, 45, 46, 0, -2, -3, -1]))
buffer = list(map(Buffer, [0, 0, 0, 0, 40, 41, 42, 43, -1, -2, 50, 51, 52, 53, 0, 0, 60, 61, 62, 63, 0, 0, 0, 0]))
# buffer = list(map(Buffer, [0, 0, 42, 43, 44, -1, -2]))

class Counter():
    STATE = 0
    
    @classmethod
    def high_energy_change(cls, source):
        def high_energy_change_observable(observer, _):
            def analyze(val):
                previous_state = cls.STATE
                cls.STATE = "high" if is_high(val) else "low"
                if cls.STATE == "high" and previous_state != cls.STATE:
                    print("change !")
                    observer.on_next(val);
                    
            source.subscribe(analyze ,observer.on_error, observer.on_completed)

        return create(high_energy_change_observable)

windows = of(*buffer).pipe(op.buffer_with_count(2, 1), op.share())

windows.pipe(
#     op.skip_last(2),
#     debug("window"),
    op.buffer_toggle(windows.pipe(Counter.high_energy_change), lambda _: windows.pipe(op.filter(is_low))),
).subscribe(print_res, print, lambda: print("completed stream"))


NameError: name 'Buffer' is not defined

In [10]:
class Counter():
    STATE = "low"
    
    @classmethod
    def my_op(cls, source):
        def new_observable(observer, _):
            def analyze(val):
                previous_state = cls.STATE
                is_low = cls.STATE == "low"
                cls.STATE = "low" if sum(val) / len(val) < 40 else "high"
                if cls.STATE != previous_state:
                    observer.on_next(cls.STATE);
                    
            source.subscribe(analyze ,observer.on_error, observer.on_completed)

        print(source)
        return create(new_observable)
#         return obs.pipe(analyze)


buffer = list(map(int, [0, 0, 0, 0, 40, 41, 42, 43, -1, -2, 50, 51, 52, 53, 0, 0, 60, 61, 62, 63, 0, 0, 0, 0]))

windows = of(*buffer).pipe(op.buffer_with_count(2, 1), op.share())
energy_change_obs = windows.pipe(Counter.my_op, debug("energy_change"), op.share())
# energy_change_obs.subscribe(print_res, print)
recording_window_openings = energy_change_obs.pipe(op.filter(lambda energy: energy == "high"))
recording_window_closings = energy_change_obs.pipe(op.filter(lambda energy: energy == "low"))

# opener = windows.pipe(Counter.my_op)
windows.pipe(
#     op.skip_last(2),
#     debug("window"),
#     op.buffer_toggle(recording_window_openings, lambda _: windows.pipe(op.filter(lambda val: sum(val) / len(val) < 40))),
    op.buffer_toggle(recording_window_openings, lambda _: recording_window_closings),

).subscribe(print_res, print, lambda: print("completed stream"))


energy_change: high
energy_change: low
res ---> [[40, 41], [41, 42], [42, 43]]
energy_change: high
energy_change: low
res ---> [[50, 51], [51, 52], [52, 53]]
energy_change: high
energy_change: low
res ---> [[60, 61], [61, 62], [62, 63]]
completed stream
